In [ ]:
#!pip install textblob 
#language recognition

In [1]:
import requests
import csv
import pandas as pd
import numpy as np
import datetime #today's date
from textblob import TextBlob 
from bs4 import BeautifulSoup
import re
import time #delay for scrapping multiple articles

In [ ]:
url = "https://www.myrhline.com/actualite-rh/de-la-gpec-et-au-workforce-planning-les-5-evolutions-a-connaitre.html"

In [2]:
df= pd.DataFrame(columns=['art_content','art_content_html','art_published_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','art_img','art_auth','art_tag']  )
df

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag


In [23]:
""" 
In: url
Out: a tuple of strings
"""

def get_all(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    art_content_html = soup.find("div", {"class": "post-detail-wrap"})
    if art_content_html is None:
        art_content_html = "no_data"

    art_content = str()
    for paragraph in art_content_html:
        art_paraph = paragraph.text.replace('\xa0', '')
        art_content += art_paraph
    if art_content is None:
        art_content = "no_data"    
   
    if soup.find("meta", {"property": "article:modified_time"}) is not None:
        
        art_published_datetime = soup.find("meta", {"property": "article:modified_time"})["content"]
        art_published_datetime = datetime.datetime.strptime(art_published_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        
    elif soup.find("meta", {"property": "article:published_time"}) is not None:
        art_published_datetime = soup.find("meta", {"property": "article:published_time"})["content"]
        art_published_datetime = datetime.datetime.strptime(art_published_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        
    else:
        art_published_datetime = datetime.datetime.now()
    
    if soup.find("meta", {"property": "og:locale"}) is not None:
        art_lang = soup.find("meta", {"property": "og:locale"})["content"]
    else:
        art_lang = "no_data"
    
    if soup.find("meta", {"property": "og:title"}) is not None:
        art_title = soup.find("meta", {"property": "og:title"})["content"]
    else:
        art_title = "no_data"
        
    if soup.find("meta", {"property": "og:url"}) is not None:
        art_url = soup.find("meta", {"property": "og:url"})["content"]
    else:
        art_url = "no_data"
    
    if soup.find("meta", {"property": "og:site_name"}) is not None:
        src_name = soup.find("meta", {"property": "og:site_name"})["content"]
    else:
        src_name = "no_data"
    
    src_type = "xpath_source" #default value  
    
    if soup.find("div", {"class": "cscra-brand dhve-mobile-logo dhve-retina-logo hav-normal-logo"}).find("a") is not None:
        src_url = soup.find("div", {"class": "cscra-brand dhve-mobile-logo dhve-retina-logo hav-normal-logo"}).find("a")["href"]
    else:
        src_url = "no_data"
    
    if soup.find("meta", {"property": "og:image"}) is not None:
        art_img = soup.find("meta", {"property": "og:image"})["content"]
    else:
        art_img = "no_data"
        
    if soup.find("meta", {"name": "author"}) is not None:   
        art_auth = soup.find("meta", {"name": "author"})["content"]
    else:
        art_auth = "no_data"

    if soup.find_all("a", {"rel": "tag"}) is not None:
        art_tags = soup.find_all("a", {"rel": "tag"})
        art_tag = []
        for tag in art_tags:
            art_tag.append(str(tag.text))
    else:
        art_tag = "no_data"
    
    
    return art_content, art_content_html, art_published_datetime, art_lang, art_title, art_url,\
        src_name, src_type, src_url, art_img, art_auth, art_tag

In [ ]:
get_all(url)[11]

'art_content' [0],'art_content_html'[1],'art_published_datetime'[2],'art_lang'[3],
    'art_title'[4],'art_url'[5],'src_name'[6],'src_type'[7],'src_url'[8],'src_img'[9],'art_auth'[10],'art_tag'[11]
    

In [4]:
"""
In: url
Out: url of the previous article
"""

def get_prev(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    prev_soup = soup.find("div", {"class": "col-sm-6 col-xs-6"}).find("a")
    if prev_soup is not None:
        return prev_soup["href"]
    else:
        return None
    
"""
In: url
Out: url of the following article
"""
    
def get_next(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    next_soup = soup.find("div", {"class": "col-sm-6 col-xs-6 text-right"})
    
    if next_soup is not None:
        next_a = soup.find("div", {"class": "col-sm-6 col-xs-6 text-right"}).find("a")["href"]
        return next_a
    else:
        return None


In [ ]:
get_prev(url)

In [ ]:
url_3 = "https://www.myrhline.com/actualite-rh/lier-management-innovant-et-rh-avec-lodyssee-manageriale.html"

In [ ]:
get_next(url_3)

In [5]:
"""
In: url 
Out: month, year of published article
"""

def art_date(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    if soup.find("meta", {"property": "article:modified_time"}) is not None:
        art_published_datetime = soup.find("meta", {"property": "article:modified_time"})["content"]
    elif soup.find("meta", {"property": "article:published_time"}) is not None:
        art_published_datetime = soup.find("meta", {"property": "article:published_time"})["content"]

    date_article = datetime.datetime.strptime(art_published_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
    return date_article   

In [ ]:
art_date(url)

In [6]:
"""
In: url, year after which the articles have to be published
Out: list of preceding articles
"""

def all_before(url, date):
    before = []
    url = get_prev(url)
    article_date = art_date(url)
    while url is not None and article_date > date:
        
        if url not in before:
            before.append(url)
            print(url, article_date)
            
        url = get_prev(url)
        
        if url is None:
            return before
        
        article_date = art_date(url)
                
    return before

"""
In: url, year after which the articles have to be published
Out: list of following articles
"""

def all_after(url, date):
    after = []
    url = get_next(url)
    article_date = art_date(url)
    while url is not None and article_date > date:
        
        if url not in after:
            after.append(url)
            print(url, article_date)
            
        url = get_next(url)

        if url is None:
            return after
        
        article_date = art_date(url)
            
    return after

In [ ]:
dummy_date = datetime.date(2020, 10, 14)

In [ ]:
date = art_date(url)

In [ ]:
date > dummy_date

In [ ]:
all_be = all_before(url, dummy_date)

In [ ]:
dummy_date_2 = datetime.date(2020, 12, 15)

In [ ]:
url_2 = "https://www.myrhline.com/actualite-rh/les-tendances-du-mobile-learning-en-france.html"

In [ ]:
all_af = all_after(url_2, dummy_date_2)

In [ ]:
allin = all_be + [url] + all_af

In [7]:
"""
In: url, year after which the articles have to be published
Out: list of all articles
"""

def all_in(url, date):
    all_be = all_before(url, date)
    all_af = all_after(url, date)
    
    return list(set(all_be + [url] + all_af))

In [8]:
url_4 = "https://www.myrhline.com/actualite-rh/actualite-sociale-2020-decryptez-lactualite-marquante-de-lannee-2020-en-presence-dexperts-en-droit-social.html"

In [9]:
date_4 = datetime.date(2020, 11, 25)

In [18]:
list_urls = all_in(url_4, date_4)

https://www.myrhline.com/actualite-rh/les-chiffres-du-marche-du-travail-fin-novembre-2020.html 2020-12-15
https://www.myrhline.com/actualite-rh/la-mobilite-professionnelle-pendant-la-crise-sanitaire.html 2020-12-11
https://www.myrhline.com/actualite-rh/conges-payes-crise-sanitaire-un-soutien-aux-entreprises-fermees.html 2020-12-11
https://www.myrhline.com/actualite-rh/la-formation-au-service-de-la-marque-employeur.html 2020-12-11
https://www.myrhline.com/actualite-rh/wingzy-lance-sa-nouvelle-ia-qui-relie-les-collaborateurs-a-distance.html 2020-12-11
https://www.myrhline.com/actualite-rh/quel-est-le-rapport-entre-les-francais-et-leur-cse.html 2020-12-28
https://www.myrhline.com/actualite-rh/comment-refuser-correctement-une-candidature.html 2020-12-11
https://www.myrhline.com/actualite-rh/comment-identifier-et-relever-les-defis-de-la-communication-et-du-management-a-distance.html 2020-12-11
https://www.myrhline.com/actualite-rh/plan-1-jeune-1-solution.html 2020-12-07
https://www.myrhline

In [19]:
list_urls

['https://www.myrhline.com/actualite-rh/la-mobilite-professionnelle-pendant-la-crise-sanitaire.html',
 'https://www.myrhline.com/actualite-rh/les-chiffres-du-marche-du-travail-fin-novembre-2020.html',
 'https://www.myrhline.com/actualite-rh/plan-1-jeune-1-solution.html',
 'https://www.myrhline.com/actualite-rh/quand-les-entreprises-se-mettent-a-la-teleconsultation-medicale.html',
 'https://www.myrhline.com/actualite-rh/la-covid-19-entrave-les-perspectives-de-carriere-chez-les-moins-de-30-ans.html',
 'https://www.myrhline.com/actualite-rh/actualite-sociale-2020-decryptez-lactualite-marquante-de-lannee-2020-en-presence-dexperts-en-droit-social.html',
 'https://www.myrhline.com/actualite-rh/conges-payes-crise-sanitaire-un-soutien-aux-entreprises-fermees.html',
 'https://www.myrhline.com/actualite-rh/la-formation-au-service-de-la-marque-employeur.html',
 'https://www.myrhline.com/actualite-rh/emploi-des-personnes-en-situation-de-handicap-6-nouvelles-mesures-annoncees.html',
 'https://www.m

In [20]:
"""
In: list of urls
Out: dataframe with all scraped articles
"""

def scrap_all(list_urls):
    i = 1
    df= pd.DataFrame(columns=['art_content','art_content_html','art_published_datetime',\
                              'art_lang','art_title','art_url','src_name','src_type','src_url',\
                              'art_img','art_auth','art_tag'])
    for url in list_urls:
        df.loc[i] = get_all(url)
        
        i+=1
    return df  

In [21]:
df2 = scrap_all(list_urls)

In [24]:
df2

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
1,La mobilité professionnelle a le vent e...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-11,fr_FR,La mobilité professionnelle pendant la crise s...,https://www.myrhline.com/actualite-rh/la-mobil...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Tendances RH, coronavirus, Covid-19,..."
2,En collaboration avec la Délégation gén...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-15,fr_FR,Chiffres du marché du travail fin novembre 2020,https://www.myrhline.com/actualite-rh/les-chif...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Recrutement, Covid-19, Dares, Emploi..."
3,Lancé en juillet 2020 par le gouverneme...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-07,fr_FR,Plan 1 jeune 1 solution,https://www.myrhline.com/actualite-rh/plan-1-j...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Recrutement, Tendances RH, 1 jeune 1..."
4,"Phénomène apparu en Amérique du Nord, l...","[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-07,fr_FR,La téléconsultation médicale en entreprise en ...,https://www.myrhline.com/actualite-rh/quand-le...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Santé, QVT, Bien-être au travail, Co..."
5,D’après une étude Sharp Europe menée au...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-28,fr_FR,Étude Sharp : Covid et télétravail freine l’am...,https://www.myrhline.com/actualite-rh/la-covid...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Gestion et Management des Talents, C..."
6,L’année 2020 est à nouveau une année sa...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-15,fr_FR,ACTUALITÉ SOCIALE 2020 : décryptez l’actualité...,https://www.myrhline.com/actualite-rh/actualit...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article sponsorisé, Droit social, droit du tr..."
7,"La ministre du Travail, de l’Emploi et ...","[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-11,fr_FR,Congés payés crise sanitaire : un soutien aux ...,https://www.myrhline.com/actualite-rh/conges-p...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Droit social, droit du travail, Tend..."
8,Un livre blanc pour comprendre les enje...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-11,fr_FR,La formation au service de la marque employeur...,https://www.myrhline.com/actualite-rh/la-forma...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Contenu téléchargeable, Marque employeur, For..."
9,"La ministre du Travail, de l’Emploi et ...","[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-28,fr_FR,Emploi des personnes en situation de handicap,https://www.myrhline.com/actualite-rh/emploi-d...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Droit social, droit du travail, AMEE..."
10,À la lecture des résultats de cette étu...,"[[ , [<i aria-hidden=""

In [25]:
"""
In: url of a random article
Out: dataframe with all scraped articles
"""

def do_it_all(url, date):
    return scrap_all(all_in(url, date))

In [26]:
df3 = do_it_all(url_4, date_4)

https://www.myrhline.com/actualite-rh/les-chiffres-du-marche-du-travail-fin-novembre-2020.html 2020-12-15
https://www.myrhline.com/actualite-rh/la-mobilite-professionnelle-pendant-la-crise-sanitaire.html 2020-12-11
https://www.myrhline.com/actualite-rh/conges-payes-crise-sanitaire-un-soutien-aux-entreprises-fermees.html 2020-12-11
https://www.myrhline.com/actualite-rh/la-formation-au-service-de-la-marque-employeur.html 2020-12-11
https://www.myrhline.com/actualite-rh/wingzy-lance-sa-nouvelle-ia-qui-relie-les-collaborateurs-a-distance.html 2020-12-11
https://www.myrhline.com/actualite-rh/quel-est-le-rapport-entre-les-francais-et-leur-cse.html 2020-12-28
https://www.myrhline.com/actualite-rh/comment-refuser-correctement-une-candidature.html 2020-12-11
https://www.myrhline.com/actualite-rh/comment-identifier-et-relever-les-defis-de-la-communication-et-du-management-a-distance.html 2020-12-11
https://www.myrhline.com/actualite-rh/plan-1-jeune-1-solution.html 2020-12-07
https://www.myrhline

In [27]:
df3

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
1,La mobilité professionnelle a le vent e...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-11,fr_FR,La mobilité professionnelle pendant la crise s...,https://www.myrhline.com/actualite-rh/la-mobil...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Tendances RH, coronavirus, Covid-19,..."
2,En collaboration avec la Délégation gén...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-15,fr_FR,Chiffres du marché du travail fin novembre 2020,https://www.myrhline.com/actualite-rh/les-chif...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Recrutement, Covid-19, Dares, Emploi..."
3,Lancé en juillet 2020 par le gouverneme...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-07,fr_FR,Plan 1 jeune 1 solution,https://www.myrhline.com/actualite-rh/plan-1-j...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Recrutement, Tendances RH, 1 jeune 1..."
4,"Phénomène apparu en Amérique du Nord, l...","[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-07,fr_FR,La téléconsultation médicale en entreprise en ...,https://www.myrhline.com/actualite-rh/quand-le...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Santé, QVT, Bien-être au travail, Co..."
5,D’après une étude Sharp Europe menée au...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-28,fr_FR,Étude Sharp : Covid et télétravail freine l’am...,https://www.myrhline.com/actualite-rh/la-covid...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Gestion et Management des Talents, C..."
6,L’année 2020 est à nouveau une année sa...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-15,fr_FR,ACTUALITÉ SOCIALE 2020 : décryptez l’actualité...,https://www.myrhline.com/actualite-rh/actualit...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article sponsorisé, Droit social, droit du tr..."
7,"La ministre du Travail, de l’Emploi et ...","[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-11,fr_FR,Congés payés crise sanitaire : un soutien aux ...,https://www.myrhline.com/actualite-rh/conges-p...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Droit social, droit du travail, Tend..."
8,Un livre blanc pour comprendre les enje...,"[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-11,fr_FR,La formation au service de la marque employeur...,https://www.myrhline.com/actualite-rh/la-forma...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Contenu téléchargeable, Marque employeur, For..."
9,"La ministre du Travail, de l’Emploi et ...","[[ , [<i aria-hidden=""true"" class=""fa fa-share...",2020-12-28,fr_FR,Emploi des personnes en situation de handicap,https://www.myrhline.com/actualite-rh/emploi-d...,myrhline.com | Actualité RH et tendances des R...,xpath_source,https://www.myrhline.com/,https://www.myrhline.com/wp-content/uploads/20...,Christophe PATTE,"[Article, Droit social, droit du travail, AMEE..."
10,À la lecture des résultats de cette étu...,"[[ , [<i aria-hidden=""